In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv')
test = pd.read_csv('/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv')
sample = pd.read_csv('/kaggle/input/e-commerce-shoppers-behaviour-understanding/sample.csv')

In [ ]:
train.head()

In [ ]:
train.dtypes

In [ ]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()

cols = ['Month_SeasonalPurchase', 'CustomerType', 'Gender', 'Cookies Setting', 'Education', 'Marital Status', 'WeekendPurchase']
train[cols] = train[cols].apply(enc.fit_transform)
test[cols] = test[cols].apply(enc.fit_transform)

In [ ]:
train.isnull().sum()

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy="mean")
train = pd.DataFrame(imp.fit_transform(train), columns=train.columns)
test = pd.DataFrame(imp.fit_transform(test), columns=test.columns)

In [ ]:
x_train = train.drop(['Made_Purchase'], axis="columns")
y_train = train['Made_Purchase']

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)
test = pd.DataFrame(scaler.fit_transform(test), columns=test.columns)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
clf = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
model = clf.fit(x_train, y_train)

print(model.best_params_)

In [ ]:
model1 = LogisticRegression(C=1.0, penalty="l2", solver="newton-cg")
model1 = model1.fit(x_train, y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf2 = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=5, n_estimators=100)
clf2 = clf2.fit(x_train, y_train)

In [ ]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
params = {
    'max_depth': [2,3,5,10,20],
    'min_samples_leaf': [5,10,20,50,100,200],
    'n_estimators': [10,25,30,50,100,200]
}

gd = GridSearchCV(estimator=rf, param_grid=params, cv = 4, n_jobs=-1, verbose=1, scoring="accuracy")

gd = gd.fit(x_train, y_train)

In [ ]:
gd.best_estimator_

In [ ]:
model2 = RandomForestClassifier(max_depth=3, min_samples_leaf=50, n_estimators=10, n_jobs=-1, random_state=42)
model2 = model2.fit(x_train, y_train)

In [ ]:
y_pred = model2.predict(test)
y_pred = y_pred.astype('bool')

In [ ]:
y_pred

In [ ]:
sample.head()

In [ ]:
submission = pd.DataFrame(columns=['id', 'Made_Purchase'])
submission['id'] = [i for i in range(len(y_pred))]
submission['Made_Purchase'] = [j for j in y_pred]
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()